In [1]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm
import json
from pprint import pprint

sys.path.append(str(Path("..").resolve()))
from src import *

# Update Queries
ℹ️ This notebook require the execution of [`1.3-Consistency_Recover`](../1-migration/1.3-Consistency_Recover.ipynb). Or you can just load the snapshop as in the following cell.

## Load post-preprocessing data
The following cell reset the `omero_museum` database loading the `4_consistent` snapshot.

In [2]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/4_consistent/omero_museum"

In [4]:
connector = MongoDBConnector("omero_museum")
db = connector.db

The collections of the omero_museum db are:
----------------------------------------
[activities]: _id capacity duration enrolled room startDate ticketIds workshopTitle
[artworks]: _id authorIds comments_star_1 comments_star_2 comments_star_3 comments_star_4 comments_star_5 date description donationState donatorId isOriginal locationName materials period sellerId size tecniques trade type
[authors]: _id birth_date gender home_town name surname
[departments]: _id floor free_spots room
[employees]: _id birth_date curriculum date_start email gender hometown name phone_number surname
[limited_events]: _id artist artworkTitles authorIds capacity description endDate roomName startDate theme ticketIds type
[messages]: _id content customerId deliveryStatus length sentDate type uri
[rooms]: _id floor
[suppliers]: _id email iban isMuseum isState name phoneNum surveys
[visitors]: _id birthDate chat comments donationIds email gender impairment isCustomer land name phoneNum saleIds surname surveys 

### UP1: _Update the description of a material_

### UP2: _Check-in a ticket_

### UP3: _Update trade information of an artwork_

### UP4: _Update information of an artwork_

### UP5: _Update curriculum of an employee_

### UP6: _Update employee in a department_

### UP7: _Update the shift of an given employee_

### UP8: _Change the date of a limited event_

### UP9: _Update a messagge in a given chat_

### UP10: _Update prices of a workshop_